# Clockodo API Testing

This notebook tests the Clockodo client functions to debug API interactions.

## Setup

First, set your environment variables with your Clockodo credentials.

In [ ]:
import os
import sys
from pathlib import Path

# Add the src directory to Python path
notebook_dir = Path.cwd()
src_dir = notebook_dir / 'src'
if str(src_dir) not in sys.path:
    sys.path.insert(0, str(src_dir))

print(f"Added {src_dir} to Python path")
print(f"Current working directory: {notebook_dir}")

In [ ]:
# Set your Clockodo credentials here
os.environ['CLOCKODO_API_USER'] = 'your.name@example.com'  # Replace with your email
os.environ['CLOCKODO_API_KEY'] = 'YOUR-API-KEY'  # Replace with your API key
os.environ['CLOCKODO_USER_AGENT'] = 'clockodo-test-notebook'
os.environ['CLOCKODO_EXTERNAL_APP_CONTACT'] = 'your.name@example.com'

print("✓ Environment variables set")

## Import Clockodo Client

In [ ]:
from clockodo_mcp.client import ClockodoClient

# Create client from environment variables
client = ClockodoClient.from_env()

print(f"Client created:")
print(f"  API User: {client.api_user}")
print(f"  Base URL: {client.base_url}")
print(f"  User Agent: {client.user_agent}")

## Test 1: List Users

In [ ]:
import json

try:
    users_response = client.list_users()
    print("✓ Successfully retrieved users")
    print(f"\nRaw response:")
    print(json.dumps(users_response, indent=2))
    
    # Parse users if available
    if 'users' in users_response:
        users = users_response['users']
        print(f"\n📊 Found {len(users)} user(s)")
        for user in users:
            print(f"  - ID: {user.get('id')}, Name: {user.get('name')}, Email: {user.get('email')}")
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()

## Test 2: Get User Reports

This is the main test for the debug tool. We'll test fetching user reports for a specific year.

In [ ]:
# Test for current year
from datetime import datetime

current_year = datetime.now().year
print(f"Testing user reports for year: {current_year}")

try:
    reports_response = client.get_user_reports(year=current_year)
    print("✓ Successfully retrieved user reports")
    print(f"\nRaw response:")
    print(json.dumps(reports_response, indent=2))
    
    # Show response structure
    print(f"\n📊 Response keys: {list(reports_response.keys())}")
    
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()

## Test 3: Get User Reports with Different Type Levels

The Clockodo API supports different type levels for aggregation.

In [ ]:
# Test different type levels (0, 1, 2, 3)
for type_level in [0, 1, 2, 3]:
    print(f"\n{'='*60}")
    print(f"Testing type_level={type_level}")
    print(f"{'='*60}")
    
    try:
        reports = client.get_user_reports(year=current_year, type_level=type_level)
        print(f"✓ Success")
        print(f"Response keys: {list(reports.keys())}")
        
        # Show a sample of the data
        print(f"\nSample data:")
        print(json.dumps(reports, indent=2)[:1000] + "..." if len(json.dumps(reports)) > 1000 else json.dumps(reports, indent=2))
        
    except Exception as e:
        print(f"❌ Error: {e}")

## Test 4: Get User Reports for Specific User

If you have multiple users, test fetching reports for a specific user.

In [ ]:
# First get the user ID from the users list
try:
    users_response = client.list_users()
    if 'users' in users_response and len(users_response['users']) > 0:
        user_id = users_response['users'][0]['id']
        print(f"Testing reports for user_id: {user_id}")
        
        reports = client.get_user_reports(year=current_year, user_id=user_id)
        print("✓ Successfully retrieved user-specific reports")
        print(f"\nResponse:")
        print(json.dumps(reports, indent=2))
    else:
        print("No users found to test with")
        
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()

## Test 5: Test Debug Tool Function

Test the actual debug tool function from the module.

In [ ]:
from clockodo_mcp.tools.debug_tools import get_raw_user_reports

print(f"Testing debug tool for year {current_year}")

try:
    result = get_raw_user_reports(year=current_year)
    print("✓ Debug tool executed successfully")
    print(f"\nResult:")
    print(json.dumps(result, indent=2))
    
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()

## Test 6: Inspect HTTP Headers

Check what headers are being sent with requests.

In [ ]:
print("Headers being sent with requests:")
headers = client.default_headers
print(json.dumps(headers, indent=2))

## Test 7: Test Different Years

Test fetching reports for different years to see what data is available.

In [ ]:
# Test last 3 years
test_years = [current_year - 2, current_year - 1, current_year]

for year in test_years:
    print(f"\n{'='*60}")
    print(f"Year: {year}")
    print(f"{'='*60}")
    
    try:
        reports = client.get_user_reports(year=year)
        print(f"✓ Success")
        
        # Try to extract some summary info
        if 'userreports' in reports:
            print(f"  Found 'userreports' key")
            print(f"  Data type: {type(reports['userreports'])}")
            if isinstance(reports['userreports'], list):
                print(f"  Number of entries: {len(reports['userreports'])}")
        else:
            print(f"  Response keys: {list(reports.keys())}")
            
    except Exception as e:
        print(f"❌ Error: {e}")